In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
setting_name = 'cs101'

exp_gen_prompt = {
    'carved_by': 'prompts_carved_by',
    'inside': 'prompts_inside',
    'painted_on': 'prompts_painted_on',
}

exp_names_ordered = [
    'carved_by',
    'inside', 
    'painted_on',
]


all_paths = {}
all_paths['ReVersion'] = {
    'custom_prompt_A': 'semantic_drift/output_{}/clip_alignment_scores_{}_{}_use_custom_prompt_objectA.csv',
    'custom_prompt_B': 'semantic_drift/output_{}/clip_alignment_scores_{}_{}_use_custom_prompt_objectB.csv',
    'full_prompt': 'semantic_drift/output_{}/clip_alignment_scores_{}_{}_use_custom_prompt_full_prompt.csv',
}

all_paths['ReVersion+TEA+0.2_0.5'] = {
    'custom_prompt_A': 'semantic_drift/output_{}_tea_0.2_0.5/clip_alignment_scores_{}_tea_0.2_0.5_{}_use_custom_prompt_objectA.csv',
    'custom_prompt_B': 'semantic_drift/output_{}_tea_0.2_0.5/clip_alignment_scores_{}_tea_0.2_0.5_{}_use_custom_prompt_objectB.csv',
    'full_prompt': 'semantic_drift/output_{}_tea_0.2_0.5/clip_alignment_scores_{}_tea_0.2_0.5_{}_use_custom_prompt_full_prompt.csv',
}

all_paths['ReVersion+TEA+0.3_0.5'] = {
    'custom_prompt_A': 'semantic_drift/output_{}_tea_0.3_0.5/clip_alignment_scores_{}_tea_0.3_0.5_{}_use_custom_prompt_objectA.csv',
    'custom_prompt_B': 'semantic_drift/output_{}_tea_0.3_0.5/clip_alignment_scores_{}_tea_0.3_0.5_{}_use_custom_prompt_objectB.csv',
    'full_prompt': 'semantic_drift/output_{}_tea_0.3_0.5/clip_alignment_scores_{}_tea_0.3_0.5_{}_use_custom_prompt_full_prompt.csv',
}

In [4]:
import os
import pandas as pd
import numpy as np

step = 1000

all_results = dict()

for exp_name in exp_names_ordered:
    all_results[exp_name] = dict()
    for method in all_paths.keys():
        print(f'---------{exp_name} {method}---------')
        all_results[exp_name][method] = dict()
        for metric in all_paths[method].keys():
            file_path = all_paths[method][metric].format(exp_name, exp_name, exp_gen_prompt[exp_name])
            if not os.path.exists(file_path):
                print(f"File {file_path} does not exist")
                continue
            # else:
            #     print(f"** File {file_path} exists")
            df = pd.read_csv(file_path)
            results = dict()
            temp = []
            for prompt_index in list(df['prompt_index'].unique()):
                prompt_df = df[df['prompt_index'] == prompt_index]
                results[prompt_index] = {
                    'prompt': prompt_df['prompt'].iloc[0],
                    'mean': prompt_df['score'].mean(),
                    'std': prompt_df['score'].std(),
                    'min': prompt_df['score'].min(),
                    'max': prompt_df['score'].max(),
                    'median': prompt_df['score'].median()
                }
                temp.append(prompt_df['score'].mean())
            all_results[exp_name][method][metric] = results
            # calculate the average over all prompts
            print(exp_name, method, metric, np.mean(temp))


metrics_list = ['custom_prompt_A', 'custom_prompt_B', 'full_prompt', 'vs_anchor_t01', 'dino_t01']
# metrics_list = ['full_prompt', 'vs_anchor_t01']


for method in all_paths.keys():
    write_str = f"{method}  & "
    for exp_name in all_results.keys():
        write_str += f"{exp_name}  & "
        if method in all_results[exp_name].keys():
            temp_str = ""
            for im, metric in enumerate(metrics_list):
                if metric in all_results[exp_name][method].keys():
                    temp = all_results[exp_name][method][metric]
                    if metric == 'dino_t01':
                        scale = 100
                    else:
                        scale = 1
                    temp = [temp[i]['mean'] * scale for i in temp.keys()]
                    avg = np.round(np.mean(temp), 2)
                    A = str(avg)
                else:
                    A = "-"
                if im == len(metrics_list) - 1:
                    temp_str += f"{A}  "
                else:
                    temp_str += f"{A}/ "
            write_str += f"{temp_str}  &  "
        else:
            temp_str = ""
            for im, metric in enumerate(metrics_list):
                if im == len(metrics_list) - 1:
                    temp_str += "- "
                else:
                    temp_str += "-/ "
            write_str += f"{temp_str}  &  "
    print(write_str)

---------carved_by ReVersion---------
carved_by ReVersion custom_prompt_A 25.636471381187437
carved_by ReVersion custom_prompt_B 15.923702639738716
carved_by ReVersion full_prompt 27.73626562754313
---------carved_by ReVersion+TEA+0.2_0.5---------
carved_by ReVersion+TEA+0.2_0.5 custom_prompt_A 27.604278798103337
carved_by ReVersion+TEA+0.2_0.5 custom_prompt_B 16.07649476687114
carved_by ReVersion+TEA+0.2_0.5 full_prompt 29.924450917243956
---------carved_by ReVersion+TEA+0.3_0.5---------
carved_by ReVersion+TEA+0.3_0.5 custom_prompt_A 27.83533891995748
carved_by ReVersion+TEA+0.3_0.5 custom_prompt_B 16.224426469802854
carved_by ReVersion+TEA+0.3_0.5 full_prompt 30.166030356089276
---------inside ReVersion---------
inside ReVersion custom_prompt_A 24.968024450302124
inside ReVersion custom_prompt_B 17.324630222320557
inside ReVersion full_prompt 27.87094946098328
---------inside ReVersion+TEA+0.2_0.5---------
inside ReVersion+TEA+0.2_0.5 custom_prompt_A 25.21718749237061
inside ReVersi

In [18]:
collect_results = {}
for method in all_paths.keys():
    write_str = f"{method}&  "
    collect_results[method] = {}
    for metric in metrics_list:
        collect_results[method][metric] = []

    for exp_name in all_results.keys():

        if method in all_results[exp_name].keys():

            for metric in metrics_list:

                if metric in all_results[exp_name][method].keys():
                    temp = all_results[exp_name][method][metric]
                    if metric == 'dino_t01':
                        scale = 100
                    else:
                        scale = 1   
                    temp = [temp[i]['mean'] * scale for i in temp.keys()]
                    avg = np.round(np.mean(temp), 2)
                    A = avg
                else:
                    A = "-"

                collect_results[method][metric].append(A)
        else:

            for metric in metrics_list:
                collect_results[method][metric].append("-")

method1s = ["EasyControl"]
method2s = ["EasyControl+TEA+0.2_0.5"]

for method1, method2 in zip(method1s, method2s):
    for metric in metrics_list:
        values1 = collect_results[method1][metric]
        values2 = collect_results[method2][metric]
        differences = []
            
        for v1, v2 in zip(values1, values2):
            # Skip if either value is not numeric
            if v1 == "-" or v2 == "-":
                continue
                
            # Calculate difference and add to list
            diff = float(v2) - float(v1)
            differences.append(diff)
        if differences:
            print('setting_name', setting_name,'Gap between', method1, method2, metric, sum(differences) / len(differences))
        else:
            print("No valid pairs for comparison")


setting_name cs101 Gap between EasyControl EasyControl+TEA+0.2_0.5 custom_prompt_A 0.08000000000000007
setting_name cs101 Gap between EasyControl EasyControl+TEA+0.2_0.5 custom_prompt_B 0.009999999999999787
setting_name cs101 Gap between EasyControl EasyControl+TEA+0.2_0.5 full_prompt 0.254999999999999
setting_name cs101 Gap between EasyControl EasyControl+TEA+0.2_0.5 vs_anchor_t01 1.6149999999999984
setting_name cs101 Gap between EasyControl EasyControl+TEA+0.2_0.5 dino_t01 2.030000000000001
